In [1]:
import pandas as pd

In [2]:
alibaba_batch_task_df = pd.read_csv('data/alibaba/batch_task_chunk_10000.csv')
alibaba_batch_task_df.head()

,task_name,instance_num,job_name,task_type,status,start_time,end_time,plan_cpu,plan_mem
0,M1,1.0,j_1,1,Terminated,419912,419912,100.0,0.20
1,R2_1,1.0,j_2,1,Terminated,87076,87086,50.0,0.20
2,M1,1.0,j_2,1,Terminated,87076,87083,50.0,0.20
3,R6_3,371.0,j_3,1,Terminated,157297,157325,100.0,0.49
4,J4_2_3,1111.0,j_3,1,Terminated,157329,157376,100.0,0.59


In [3]:
alibaba_batch_task_df = alibaba_batch_task_df[alibaba_batch_task_df['status'] == 'Terminated']
alibaba_batch_task_df.head()

,task_name,instance_num,job_name,task_type,status,start_time,end_time,plan_cpu,plan_mem
0,M1,1.0,j_1,1,Terminated,419912,419912,100.0,0.20
1,R2_1,1.0,j_2,1,Terminated,87076,87086,50.0,0.20
2,M1,1.0,j_2,1,Terminated,87076,87083,50.0,0.20
3,R6_3,371.0,j_3,1,Terminated,157297,157325,100.0,0.49
4,J4_2_3,1111.0,j_3,1,Terminated,157329,157376,100.0,0.59


In [4]:
alibaba_batch_task_df = alibaba_batch_task_df.sort_values(by='start_time', ascending=True)
alibaba_batch_task_df.reset_index(drop=True, inplace=True)
alibaba_batch_task_df.head()

,task_name,instance_num,job_name,task_type,status,start_time,end_time,plan_cpu,plan_mem
0,M2,40.0,j_1661,1,Terminated,86207,86485,50.0,0.39
1,J7_1_2_6,1.0,j_1661,1,Terminated,86207,86622,50.0,0.20
2,M3,1.0,j_1661,1,Terminated,86207,86210,50.0,0.20
3,R4_3,1.0,j_1661,1,Terminated,86207,86212,50.0,0.20
4,M5_4,17.0,j_1661,1,Terminated,86207,86300,50.0,0.39


In [5]:
status_categories = alibaba_batch_task_df['status'].unique()
print(status_categories)
alibaba_batch_task_df = alibaba_batch_task_df.drop(columns=['status','job_name'])
alibaba_batch_task_df.head()


['Terminated']


,task_name,instance_num,task_type,start_time,end_time,plan_cpu,plan_mem
0,M2,40.0,1,86207,86485,50.0,0.39
1,J7_1_2_6,1.0,1,86207,86622,50.0,0.20
2,M3,1.0,1,86207,86210,50.0,0.20
3,R4_3,1.0,1,86207,86212,50.0,0.20
4,M5_4,17.0,1,86207,86300,50.0,0.39


In [6]:
def determine_task_category(task_name):
    if task_name == 'MergeTask':
        return '1' #merge
    elif task_name.startswith('M'):
        return '2'
    elif task_name.startswith('J'):
        return '3'
    elif task_name.startswith('R'):
        return '4'
    else:
        return '0' #independent

alibaba_batch_task_df['task_category'] = alibaba_batch_task_df['task_name'].apply(determine_task_category)
alibaba_batch_task_df.head()

,task_name,instance_num,task_type,start_time,end_time,plan_cpu,plan_mem,task_category
0,M2,40.0,1,86207,86485,50.0,0.39,2
1,J7_1_2_6,1.0,1,86207,86622,50.0,0.20,3
2,M3,1.0,1,86207,86210,50.0,0.20,2
3,R4_3,1.0,1,86207,86212,50.0,0.20,4
4,M5_4,17.0,1,86207,86300,50.0,0.39,2


In [7]:
# Split task_name by "_" and extract the numbers
alibaba_batch_task_df['task_name'] = alibaba_batch_task_df['task_name'].str.replace('[RJM]', '', regex=True)
alibaba_batch_task_df['numbers'] = alibaba_batch_task_df['task_name'].apply(lambda x: [int(num) for num in x.split('_') if num.isdigit()])

# Create columns for job_1, job_2, job_3...
for i in range(1, 60):
    col_name = f'job_{i}'
    alibaba_batch_task_df[col_name] = alibaba_batch_task_df['numbers'].apply(lambda x: 2 if x and x[0] == i else (1 if i in x else 0))

alibaba_batch_task_df.drop(columns=['numbers'], inplace=True)
alibaba_batch_task_df.drop(columns=['task_name'], inplace=True)

alibaba_batch_task_df.head()

,instance_num,task_type,start_time,end_time,plan_cpu,plan_mem,task_category,job_1,job_2,job_3,...,job_50,job_51,job_52,job_53,job_54,job_55,job_56,job_57,job_58,job_59
0,40.0,1,86207,86485,50.0,0.39,2,0,2,0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,1,86207,86622,50.0,0.20,3,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1.0,1,86207,86210,50.0,0.20,2,0,0,2,...,0,0,0,0,0,0,0,0,0,0
3,1.0,1,86207,86212,50.0,0.20,4,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,17.0,1,86207,86300,50.0,0.39,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
alibaba_batch_task_df['instance_num'] = alibaba_batch_task_df['instance_num'].astype(int)
alibaba_batch_task_df.to_csv('data/alibaba/batch_task_chunk_preprocessed_10000.csv', index=False)